<a href="https://colab.research.google.com/github/tiendatngcs/NLP/blob/master/Data_cleaning_Vietnamese_news_topic_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pickle as pkl
import string
import random
import re

In [23]:
with open("/content/drive/My Drive/corpus.pkl", "rb") as infile:
  corpus_dict = pkl.load(infile)

In [24]:
topic_list = [
              "business",
              "entertainment",
              "sports",
              "laws",
              "education",
              "health",
              "life",
              "travel",
              "science",
              "digital",
              "vehicles",
              "comedy",
]
topic_dict = dict(enumerate(topic_list))

In [25]:
print("Topic: Number of articles")
for (key, value) in corpus_dict.items():
  print("{}: {}".format(topic_dict[key], len(value)))

Topic: Number of articles
business: 550
entertainment: 499
sports: 560
laws: 43
education: 35
health: 520
life: 545
travel: 468
science: 33
digital: 500
vehicles: 44
comedy: 35


In [26]:
corpus_dict[3]

['"Mẹ đừng có thương người nhầm chỗ nữa", con trai bà Tần ngồi cuối phòng xử án của TAND Hà Nội vò đầu khẽ nói khi nghe thấy bà xin HĐXX không bỏ tù bị cáo, ngày 20/8. Bà giáo già vuốt lại nếp tóc bạc trắng đầu, quay sang con trai phân trần: "Em nó còn 3 đứa con nhỏ, mẹ không đành". Sáng 21/8, Giang, 39 tuổi, cựu giám đốc Công ty TNHH MTV chứng khoán ngân hàng Đông Á (DAS) bị TAND Hà Nội tuyên phạt 20 năm tù về hai tội Thao túng giá chứng khoán và Lừa đảo chiếm đoạt tài sản. Trong hai ngày xét xử, khi đi qua cô giáo cũ sau hai năm tránh mặt, trốn nợ rồi bị bắt, Giang không chút biểu cảm. Trước đó, họ từng nhiều năm coi nhau như người nhà. Vợ chồng bà Tần bị lừa tiền vẫn xin HĐXX cho bị cáo Giang hưởng khoan hồng. Ảnh: Lam Vân 50 năm trước, bà Tần tốt nghiệp ngành sư phạm Văn. Trong khoá học sinh đầu tiên bà làm chủ nhiệm suốt 4 năm có mẹ của Giang. Bà Tần cưng cô trò thông minh này, ngày thường đến chơi, ngày cưới đến dự. Năm 1981, ngày Giang ra đời, bà Tần không có xe đạp, vẫn đi bộ m

In [28]:
selected_corpus_list = []
selected_topic_list = []
for (key, value) in corpus_dict.items():
  if len(value)> 300:
    selected_topic_list.append(topic_dict[key])
    selected_corpus_list.append(value)
selected_corpus_dict = dict(enumerate(selected_corpus_list))

We select classes that has decent amount of data for training and testing.

In [29]:
selected_corpus_dict.keys()
selected_topic_list

['business', 'entertainment', 'sports', 'health', 'life', 'travel', 'digital']

In [56]:
# Fetch list of Vietnamese stop words, credit to Van-Duyet Le https://github.com/duyet
import requests
stop_words = requests.get("https://raw.githubusercontent.com/stopwords/vietnamese-stopwords/master/vietnamese-stopwords.txt").text.split("\n")

In [42]:
def train_test_split(corpus_dict: dict,
                     train_size: int,
                     shuffle: bool = False,
                     random_seed: int = None,):
  # split into train-test
  train_dict = {}
  test_dict = {}
  for (key, value) in corpus_dict.items():
    train_dict[key] = value[:train_size]
    test_dict[key] = value[train_size:]
  return train_dict, test_dict

def clean_data(corpus_dict: dict):
  '''
  This method performs two tasks:
  * Join the list of strings for each class.
  * Clean the text data for each class.
  '''
  # join
  for (key, value) in corpus_dict.items():
    corpus_dict[key] = ' '.join(value)
  # clean text data
  for (key, value) in corpus_dict.items():
    corpus_dict[key] = corpus_dict[key].lower()  # lowercase
    corpus_dict[key] = re.sub('[%s]' % re.escape(string.punctuation), '', corpus_dict[key])  # remove punctuations
    corpus_dict[key] = re.sub('\w*\d\w*', '', corpus_dict[key])
    corpus_dict[key] = re.sub('[‘’“”…]', '', corpus_dict[key])
    corpus_dict[key] = re.sub('\n', ' ', corpus_dict[key])
    corpus_dict[key] = re.sub('\s\s+', ' ', corpus_dict[key])
  return corpus_dict
  
  

Let's select 300 articles from each selected topics for training. The rest are for testing.

In [44]:
train_dict, test_dict = train_test_split(selected_corpus_dict, train_size=300, )
[cleaned_train_dict, cleaned_test_dict] = [clean_data(corpus_dict) for corpus_dict in [train_dict, test_dict]]

In [45]:
cleaned_train_dict[0]

'tôm hùm bông đặc sản quy nhơn năm ngoái được bán triệu đồng và hiếm khi có giá dưới một triệu đồng một kg nay chỉ còn đồng thậm chí một số người nuôi phải bán theo hình thức cân xô vì giá giảm quá sốc ông hòa người nuôi khoảng con tôm hùm tại nhơn hải quy nhơn cho biết đây là mức thấp nhất từ khi vào nghề tới nay đợt dịch thứ hai khiến sức mua tại các nhà hàng quán ăn ở đà nẵng và tp hcm giảm sút mặt khác các đơn vị kinh doanh bất động sản xây dựng đòi lại mặt nước khiến người dân phải bán tháo tôm hùm bông ảnh thế giới hải sản cùng với tôm hùm bông cá mú – loại chuyên được bán trong các nhà hàng quán cũng rớt giá thê thảm nếu trước đây loại này được bán tại vựa với giá lên tới đồng một kg và tại nhà hàng khoảng đồng thì nay chỉ còn đồng người nuôi tại cam ranh cho biết họ đang lỗ khoảng đồng một kg đây cũng là mức giá thấp nhất trong năm qua ông hạnh ở cam ranh cho biết lượng cá mú trong ao đã đạt trọng lượng kg nhưng vẫn chưa xuất bán được ông và các hộ nuôi cũng đang thua lỗ nên bu

Convert data dictionary to pandas DataFrame